# proAnubis Data Analysis Tutorial

## Prerequisites
You’ll need to know a bit of Python. For a refresher, see the [Python tutorial](https://docs.python.org/3/tutorial/). Furthermore, you should know something about the detector i.e. what is RPC, what is TDC...

### Osiris Version
Osiris conntains all the function for extracting data from a raw files.  
The version of Osiris is slightly different to Michael's version. This version contains the realigner built into the fReader and some extra functions (skip_events).  
Hence please use the version of Osiris I have provided. For versions compatible to the original Osiris, consult the labnotes.

### Python Version
Python 3.x (Ensure you have a compatible version of Python 3.x installed. Python 2.x is not supported.)


### Required Libraries and Modules
The following libraries and modules are required to run the provided code:



In [4]:
!pip install numpy hist matplotlib mplhep pandas plotly scipy tqdmf

In [2]:
import importlib
import sys
from tqdm import tqdm
import  os
import glob
import pickle
# Add the directories to the sys.path
dir_path = "C://Users//jony//Programming//Python//Anubis//anubis//" # insert your directory path
sys.path.append(dir_path + "Osiris//processing//python")
sys.path.append(dir_path + "tools")

import Analysis_tools as ATools
import proAnubis_Analysis_Tools
import Reconstruction_tools as RTools
import Overview_tools as overview
import mplhep as hep
import Visual_tools as VTools
import Timing_tools as TTools
import rawFileReader
import Correction_tools as CTools

hep.style.use([hep.style.ATLAS])

# Specify the directory
all_files = sorted([f for f in os.listdir("data") if os.path.isfile(os.path.join("data", f))], reverse=True) ##all files in data directory sorted from the newest to the oldest
print(all_files)

['vertices.pkl', 'tot_std.pkl', 'tot_mean.pkl', 'supervertices.pkl', 'straight.pkl', 'proAnubis_240627_1720.raw', 'hvScan.csv', 'example_chunks.pkl', 'dataAtlas.root']


I would recommend to store all your data raw files in a one directory, let's say "data" and then choose it on the run. The code above gives you a list of all the files in the folder.

### Tutorial Coverage

This is a quick start tutorial example. In this tutorial you will find

1. Example usage of various functions, with high level explainations
2. How data gathered from rawfileReader, how data are processed with each other
3. Reproducing all essential results obtained so far
4. Comments on limitations and warning for usage
5. Tutorial on Reconstructor, Tracks, Time_nalyser and Realigner

<span style="color:green">For a documentation styled explaination for each function, see other notebook</span>.

### Usage Warning
The tutorial here was tested on the file "example_chunks.pkl". There are some known issues such as misalignement after a long time or over fitting in the optimasation steps for a vertex. Every code has bugs, so if you will find them, correcte them, message me or ask Michael. 

# Data Extraction
The data, which we are getting from TDC (Time Digital Converter), are firstly in raw files. You can get them either from the directory on the HEP network `/r04/atlas/revering/data` and choose the time of the data. Alternatively, you can ask Michael to send you the file. Then, you can use `rawFilReader` to extract data into `proAnubEvent`, which contains `tdc5Events` (actual useful data). The Pro_Anubis is run using triggers, which triggers data taking for 1250ns if there are 4 eta side strips triggered within a fixed time frame.

Events are collected in chunks of 100 events. This is due to alignment proccess. 

In [20]:
file_name = "proAnubis_240627_1720.raw"
fReader = rawFileReader.fileReader(dir_path+"data//"+file_name) # load in the classs object    
max_chunk_num = 1000
chunk_num = 0
example_chunks = []
with tqdm(total=max_chunk_num, desc=f"Processing Chunks {file_name}", unit="Chunks") as pbar:   
    while chunk_num < max_chunk_num:
        chunk = fReader.get_aligned_events()
        example_chunks.append(chunk)
        chunk_num += 1
        pbar.update(1)
print(chunk[0].tdcEvents)
    

Processing Chunks proAnubis_240627_1720.raw: 100%|██████████| 1000/1000 [00:10<00:00, 96.08Chunks/s] 

[<rawEventBuilder.tdcEvent object at 0x000001C11CD22530>, <rawEventBuilder.tdcEvent object at 0x000001C11CD22B60>, <rawEventBuilder.tdcEvent object at 0x000001C11CD23340>, <rawEventBuilder.tdcEvent object at 0x000001C11CD21660>, <rawEventBuilder.tdcEvent object at 0x000001C11CD21E70>]


### tdcEvent Class

#### Overview
The `tdcEvent` class is what the output from rawfileReaders are. It is basically the most fundamental high-level data unit which we have.
It consists of `header`, `words`, `EOB`, but the most important bits are words, which represent individual hits (i.e. information abouth TDC, channel, time...). 
The class also contains timestamp, based on the trigger system.
#### Class Variables

##### `header`
- **Type:** `Any`
- **Description:** Represents the header word of the event. 

##### `words`
- **Type:** `List[Any]`
- **Default:** `[]`
- **Description:** A list containing words for each event. These words represent the hits and their information about tdc, channel time...

##### `time`
- **Type:** `Any`
- **Description:** Holds the timing information for the event, a time stamp in Daytime format. Managed by a trigger system

##### `EOB`
- **Type:** `Any`
- **Default:** `None`
- **Description:** Represents the end of block indicator. This is used to signify the end of a data block within a trigger.

##### `qual`
- **Type:** `int`
- **Default:** `0`
- **Description:** A quality check indicator. The `qual` value provides information on data integrity:
  - If `qual == 0`, it means the data has not experienced corruption that required corrections in the raw file reader.
  - If `qual != 0`, it indicates that data corruption occurred and was captured by the system.


The whole process is written in a one function in `Overview_tools`, where most of the basic tools are. It can also skip the events to some specific time and read for some specific time, return fReader and in the future it would be able to read tdc5 events.

In [23]:
importlib.reload(overview)
example_chunks, times, fReader = overview.get_chunks(file_name)

Initial time 2024-06-27 16:20:18.087426 1719501618.087426


Processing Chunks proAnubis_240627_1720.raw: 100%|██████████| 200/200 [00:02<00:00, 91.59Chunks/s] 

Ending time: 2024-06-27 16:20:22.062693
Number of chunks: 200


However, the data extraction normaly takes a long time, and it is performance bottleneck, for that reason it is much better to save the chunks into `pkl` file and unpickle when you need them. But do not make the pickle file to large, otherwise it will slow down the program again.   

In [ ]:
#Save
with open(dir_path + "data//example_chunks.pkl", "wb") as f:
    pickle.dump(example_chunks, f)

In [3]:
#Read
with open(dir_path + "data//example_chunks.pkl", "rb") as f:
    example_chunks = pickle.load(f)
print(example_chunks[0])

[<rawEventBuilder.proAnubEvent object at 0x000001AF9CE250C0>, <rawEventBuilder.proAnubEvent object at 0x000001AF9CE0D150>, <rawEventBuilder.proAnubEvent object at 0x000001AFBEC6D750>, <rawEventBuilder.proAnubEvent object at 0x000001AFBEC6DAB0>, <rawEventBuilder.proAnubEvent object at 0x000001AFBEC6DDB0>, <rawEventBuilder.proAnubEvent object at 0x000001AFBEC6E1D0>, <rawEventBuilder.proAnubEvent object at 0x000001AFBEC6E410>, <rawEventBuilder.proAnubEvent object at 0x000001AFBEC6E650>, <rawEventBuilder.proAnubEvent object at 0x000001AFBEC6E890>, <rawEventBuilder.proAnubEvent object at 0x000001AFBEC6EAD0>, <rawEventBuilder.proAnubEvent object at 0x000001AFBEC6ED70>, <rawEventBuilder.proAnubEvent object at 0x000001AFBEC6EFB0>, <rawEventBuilder.proAnubEvent object at 0x000001AFBEC6F1F0>, <rawEventBuilder.proAnubEvent object at 0x000001AFBEC6F490>, <rawEventBuilder.proAnubEvent object at 0x000001AFBEC6F6D0>, <rawEventBuilder.proAnubEvent object at 0x000001AFBEC6F970>, <rawEventBuilder.proAnu

In [49]:
importlib.reload(CTools)
corrector = CTools.Tanalyser()
all_files = sorted([f for f in os.listdir("data/chunks") if os.path.isfile(os.path.join("data/chunks", f))], reverse=True) ##all files in data directory sorted from the newest to the oldest
with tqdm(total=len(all_files), desc="Processing Chunks", unit="Files") as pbar:   
    for file_name in all_files:
        with open(dir_path + "data//chunks//"+file_name, "rb") as f:
            example_chunks = pickle.load(f)
        corrector.calculate_tot(example_chunks)
        pbar.update(1)
corrector.plot_tot()

Processing Chunks: 100%|██████████| 100/100 [49:10<00:00, 29.51s/Files]


In [50]:
#save
with open(dir_path + "data//mean.pkl", "wb") as f:
    pickle.dump(corrector.tot_mean, f)
with open(dir_path + "data//std.pkl", "wb") as f:
    pickle.dump(corrector.tot_std, f)

In [48]:
with open(dir_path + "data//mean.pkl", "rb") as f:
    tot_mean = pickle.load(f)
with open(dir_path + "data//std.pkl", "rb") as f:
    tot_std = pickle.load(f)

importlib.reload(CTools)
corrector = CTools.Tanalyser()
corrector.tot_mean = tot_mean
corrector.tot_std = tot_std


corrector.plot_tot()

In [46]:
print(corrector.tot_mean)
print(corrector.tot_std)

[[[13.         13.         13.         ... 13.         13.
   13.        ]
  [11.19791667 12.109375   12.109375   ... 13.         13.
   13.        ]
  [12.23958333 12.5        13.28125    ... 13.         13.
   13.        ]
  ...
  [ 1.171875    1.5625     13.         ...  8.59375    13.
   13.        ]
  [13.         13.          2.67857143 ... 13.         13.
   13.        ]
  [ 0.52083333 13.         13.         ... 13.         13.
   13.        ]]

 [[13.         13.         13.         ... 13.         13.
   13.        ]
  [13.         14.453125   14.375      ... 13.         13.
   13.        ]
  [12.5        14.453125   15.0390625  ... 13.         13.
   13.        ]
  ...
  [13.         13.          3.90625    ...  8.33333333  9.375
   13.        ]
  [13.         13.          4.21875    ...  9.89583333 13.
   13.        ]
  [13.          1.5625      2.734375   ... 13.         13.
   13.        ]]

 [[13.         13.         13.         ... 13.         13.
   13.        ]
  [12.

### Alignment Algorithm
We find out that the tdcs sometimes miss events, and the resultant rpc events are uncorrelated. To fix that, we monitor alignment metric, and in the moment when it surpasses some limit, we correct it. The code is in the `rawEventBuilder` along `Analysis_tools`

The code bellow plots the alignment metric. Again, it is in `Overview_tools`.

In [28]:
importlib.reload(overview)
mets = overview.alignment(example_chunks)
max(mets[0])

Alignment Done


25.98510764039865

Another important metric related to alignment is the ratio of off peaks hit times. The right time distribution looks something like this:

In [32]:
importlib.reload(overview)
overview.hit_time_hist(example_chunks)

Percentage of hits >1200: 0.03200925729365684
Hit Time Histogram Done


([[(array([   0.  ,   16.63,   33.26,   49.89,   66.52,   83.15,   99.78,
            116.41,  133.04,  149.67,  166.3 ,  182.93,  199.56,  216.19,
            232.82,  249.45,  266.08,  282.71,  299.34,  315.97,  332.6 ,
            349.23,  365.86,  382.49,  399.12,  415.75,  432.38,  449.01,
            465.64,  482.27,  498.9 ,  515.53,  532.16,  548.79,  565.42,
            582.05,  598.68,  615.31,  631.94,  648.57,  665.2 ,  681.83,
            698.46,  715.09,  731.72,  748.35,  764.98,  781.61,  798.24,
            814.87,  831.5 ,  848.13,  864.76,  881.39,  898.02,  914.65,
            931.28,  947.91,  964.54,  981.17,  997.8 , 1014.43, 1031.06,
           1047.69, 1064.32, 1080.95, 1097.58, 1114.21, 1130.84, 1147.47,
           1164.1 , 1180.73, 1197.36, 1213.99, 1230.62, 1247.25, 1263.88,
           1280.51, 1297.14, 1313.77, 1330.4 , 1347.03, 1363.66, 1380.29,
           1396.92, 1413.55, 1430.18, 1446.81, 1463.44, 1480.07, 1496.7 ,
           1513.33, 1529.96, 1546.59, 

There are some times outside the main peak (defined as time_range = (150,350)), and when there is something wrong with TDCs, it is projected into this metric. Generally, we are aiming around 0.15 value. If something is wrong, look at absolute "bad" hits metric.

In [37]:
importlib.reload(overview)
time_hits, bvg = overview.tdc_times_stats(example_chunks)
overview.bvg(bvg)
overview.abs_bvg_hits(example_chunks)

BVG Done
Absolute BVG Done


[[[280,
   252,
   278,
   257,
   259,
   251,
   271,
   270,
   280,
   335,
   313,
   249,
   237,
   285,
   251,
   233,
   331,
   231,
   272,
   251,
   273,
   243,
   244,
   245,
   260,
   244,
   257,
   307,
   283,
   246,
   277,
   246,
   281,
   258,
   329,
   286,
   257,
   291,
   248,
   252,
   249,
   233,
   236,
   259,
   256,
   263,
   289,
   249,
   298,
   249,
   271,
   284,
   277,
   247,
   246,
   261,
   243,
   253,
   258,
   267,
   255,
   271,
   249,
   274,
   266,
   260,
   258,
   261,
   251,
   263,
   260,
   246,
   233,
   277,
   256,
   291,
   293,
   289,
   264,
   249,
   242,
   261,
   293,
   239,
   273,
   247,
   259,
   219,
   248,
   282,
   279,
   250,
   284,
   254,
   262,
   261,
   342,
   248,
   254,
   255,
   239,
   292,
   231,
   288,
   251,
   329,
   250,
   261,
   233,
   259,
   253,
   280,
   282,
   285,
   286,
   271,
   256,
   255,
   257,
   278,
   230,
   236,
   260,
   282,
   303,


There are many more functions in overview, which are worth knowing about such as cluster size, hit count, efficiency of reconstruuction and many more. Generally, if you will need to have some general metric, there is some chance it might have been already implemented. For better understanding of one particular function just look at the code.

The code above prints all the important metrics into pdf, so you can have quick scan of a file if everything is alrgith.

In [26]:
#Jupyter notebook doesn't reload your import even when the content of the file is changed. This is crucial
importlib.reload(overview)
from matplotlib.backends.backend_pdf import PdfPages
file_name = "proAnubis_240627_1720"
with PdfPages(f"{file_name}.pdf") as pdf:
    overview.alignment(example_chunks, pdf = pdf)
    time_hits, bvg = overview.tdc_times_stats(example_chunks)
    overview.bvg(bvg, pdf = pdf)
    overview.abs_bvg_hits(example_chunks, pdf = pdf)
    overview.hit_time_hist(example_chunks, pdf = pdf)
    overview.hit_channel_hist(time_hits, pdf = pdf)
    overview.efficiency(example_chunks, pdf = pdf)


LookupError: An associated PostScript font (required by Matplotlib) could not be found for TeX font 'zptmcm7y' in 'C:/Users/jony/AppData/Local/MiKTeX/fonts/map/pdftex/pdftex.map'; this problem can often be solved by installing a suitable PostScript font package in your TeX package manager

## Conclusion

fReader doesn't have many instances for storing metric, they are typically thrown out after use, hence it is very difficult to extract these out. However, it becomes easier as we move to Timing Analyser and Reconstructor where the class instances can be called directly to extract and record data. 

The general idea is, if the data is in Osiris, they are NOT stored so that it will run smoothly always. Other data like reconstructio and timing analyser will be stored in class instances, which will remain in memory as long you as you don't reload them

# The Reconstructor
Probably the most important thing which we want to know about the detected particles are their trajectories, if there was any vertex and time of flight information. All these functions are consolidated in the `Reconstructor` function in `Reconstruction_tools.py`. Reconstructor provides the following functionality:
- forms clusters out of the individual hits (produces `Cluster` objects)
- makes tracks through these clusters (`Track` class instances)
- if two tracks are detected in a one `proAnubis_evt`, it is handled like a vertex and `Vertex` object (`Vertex` class is children of `Track` class) is produced

Individual objects encapsulate all the information which you need about them, for instance, tracks have all the information about chi2, time of flight, clusters used, clusters have their time information, position uncertainty etc. OOP is very useful in this way.

In [8]:
importlib.reload(RTools)
importlib.reload(VTools)

reconstructor = RTools.Reconstructor()
max_chunk_num = 100
file_name = "proAnubis_240627_1720"
with tqdm(total=max_chunk_num, desc=f"Processing Chunks {file_name}", unit="Chunks") as pbar:
    for chunk in example_chunks[:max_chunk_num]:
        chunk_tracks = reconstructor.reconstruct_tracks(chunk) #you can also access the tracks as inner variable
        for event_num, event_tracks in enumerate(chunk_tracks):
            if not event_tracks:
                continue
            track = event_tracks[0]
            if isinstance(track, RTools.Vertex):
                #if it is vertex, show me
                VTools.event_3d_plot(chunk[event_num], "Example Vertex")
        reconstructor.tracks = [] #clear the tracks
        pbar.update(1)
VTools.event_3d_plot(chunk[event_num], "Example event") #show me the last event
print("Coordinates:", track.coordinates)
print("Is it cosmic?", track.cosmic)
print("Direction:", track.direction) #centroid and direction specifies the track geometry
print("Centroid", track.centroid)


            
        


Processing Chunks proAnubis_240627_1720:  18%|█▊        | 18/100 [00:00<00:01, 43.82Chunks/s]

rpcHit(channel=16, time=191.40625, eta=True, event_num=0, rpc=0)
rpcHit(channel=18, time=191.40625, eta=True, event_num=0, rpc=0)
rpcHit(channel=38, time=182.8125, eta=False, event_num=0, rpc=0)
rpcHit(channel=41, time=182.03125, eta=False, event_num=0, rpc=0)
rpcHit(channel=16, time=190.625, eta=True, event_num=0, rpc=1)
rpcHit(channel=18, time=191.40625, eta=True, event_num=0, rpc=1)
rpcHit(channel=38, time=180.46875, eta=False, event_num=0, rpc=1)
rpcHit(channel=39, time=180.46875, eta=False, event_num=0, rpc=1)
rpcHit(channel=42, time=178.90625, eta=False, event_num=0, rpc=1)
rpcHit(channel=15, time=189.84375, eta=True, event_num=0, rpc=2)
rpcHit(channel=17, time=191.40625, eta=True, event_num=0, rpc=2)
rpcHit(channel=39, time=178.90625, eta=False, event_num=0, rpc=2)
rpcHit(channel=42, time=178.90625, eta=False, event_num=0, rpc=2)
rpcHit(channel=7, time=189.0625, eta=True, event_num=0, rpc=3)
rpcHit(channel=4, time=190.625, eta=True, event_num=0, rpc=3)
rpcHit(channel=49, time=18

Processing Chunks proAnubis_240627_1720: 100%|██████████| 100/100 [00:08<00:00, 12.27Chunks/s]


rpcHit(channel=58, time=192.1875, eta=False, event_num=0, rpc=1)
rpcHit(channel=14, time=465.625, eta=False, event_num=0, rpc=3)
rpcHit(channel=55, time=195.3125, eta=False, event_num=0, rpc=3)
rpcHit(channel=53, time=199.21875, eta=False, event_num=0, rpc=4)
rpcHit(channel=5, time=204.6875, eta=True, event_num=0, rpc=5)
rpcHit(channel=8, time=213.28125, eta=True, event_num=0, rpc=5)
rpcHit(channel=9, time=212.5, eta=True, event_num=0, rpc=5)
rpcHit(channel=10, time=211.71875, eta=True, event_num=0, rpc=5)
rpcHit(channel=26, time=203.125, eta=False, event_num=0, rpc=5)
rpcHit(channel=27, time=203.125, eta=False, event_num=0, rpc=5)
rpcHit(channel=28, time=203.90625, eta=False, event_num=0, rpc=5)
rpcHit(channel=29, time=203.90625, eta=False, event_num=0, rpc=5)
rpcHit(channel=30, time=203.125, eta=False, event_num=0, rpc=5)
rpcHit(channel=32, time=203.125, eta=False, event_num=0, rpc=5)
rpcHit(channel=33, time=199.21875, eta=False, event_num=0, rpc=5)
rpcHit(channel=34, time=198.4375, 

# The `Timing Analyser`

The `Timing_Analyser` class is designed to process and analyze timing data from event chunks. This class helps in calculating and visualizing time of flight analysis, residuals, and other related metrics for the pro_anubis detectors. It provides functionalities to update events, read TDC (Time-to-Digital Converter) time differences, calculate residuals, check eta trigger, and plot various data for analysis.

When using any class from `proAnubis_analysis_tools`, it is important that a new event loop format is used. The loop involves an initialisation, which clears all instances. When the event loop is entered, calculations are done through internal instances, meaning one must not TAnalyser, but to update the event_chunk through TAnalyser.`update_event`

One of the most useful metric to determine the corruption status is TAnalyser.`check_eta_trigger`. This function checks if each chunk has corruption and also count the total number of corrupted events with which RPC was involved in the corruption within the class instance

In [17]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
interval = 100 # Set your monitoring chunck size
fReader = rawFileReader.fileReader(file_path[0]) # load in the classs object
fReader.skip_events(350000) # Skip the first 1000 events
order = [[0,1], [1,2], [2,3], [3,4]] # Order what you want to align
max_process_event_chunk = 15000 # End the loop early
processedEvents = 0 # Initialisation
initial_event_chunk = fReader.get_aligned_events(order=order, interval=interval)
TAnalyser = proAnubis_Analysis_Tools.Timing_Analyser(initial_event_chunk,processedEvents)
with tqdm(total=max_process_event_chunk, desc="Processing Events", unit='Events') as pbar:
    while processedEvents < max_process_event_chunk:
        processedEvents += 1
        event_chunk = fReader.get_aligned_events(order=order, interval=interval)
        if event_chunk:
            TAnalyser.update_event(event_chunk, processedEvents)
            status, failure = TAnalyser.check_eta_trigger()
            if not status:
                #print(f'Event chunk {processedEvents} failed the eta trigger check')
                #print(failure)
                pass
        pbar.update(1)

Skipping Events: 350002Events [00:08, 39137.30Events/s]                        
Processing Events:  65%|██████▌   | 9796/15000 [02:55<01:33, 55.76Events/s]


KeyboardInterrupt: 

To plot these on a graph, note we are calling from TAnalyser to obtain the information we needed

In [55]:
event_counts_windows = [len(TAnalyser.count[count]) for count in range(7)]
total_windows = sum(event_counts_windows)
normalized_windows = [count / total_windows for count in event_counts_windows]
normalized_windows.append(normalized_windows[-1])
plt.figure(figsize=(12, 8))
r1 = list(range(7)) + [6.5]

# plt.step(r1, normalized_linux, color='mediumseagreen', linestyle='-', linewidth=2, markersize=6, label='0-15000', alpha=1, where='mid')
plt.step(r1, normalized_windows, color='dodgerblue', linestyle='-', linewidth=2, markersize=6, label='0-15000?', alpha=1, where='mid')

plt.xlabel('Trigger Number')
plt.ylabel('Number of Events Normalized')
plt.title('Normalized Event Count')
plt.ylim(0)
plt.xlim(0, 6.5)
plt.xticks(range(7))
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.legend()
plt.show()

As an example, one can also plot directly from class accessing the class variable. This makes no difference fundamentally, but it does make the code much clearer and immediately clear what you are plotting

In [50]:
TAnalyser.plot_rpc_involvement_histogram()

## TDC monitoring
The TDC monitoring script is innatly written in the fReader. Every 2500 events, the TDC status is monitored. This TDC status metric counts the number of events where the first time is within the "bad region" with an event time> 300 ns compare to the "good region" where the event time <300ns. It was typicaly found that the TDC is in an error state when the fraction is larger than 0.2

As a complementary output, TDC_info is also outputted to show the first hit time and first hit channels on each TDC.

In [59]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
fReader = rawFileReader.fileReader(file_path[0]) # load in the classs object
interval = 100 # Set your monitoring chunck size
event_counter = 100*interval
#event_counter += fReader.skip_events(3500*interval) # Skip the first 1000 events
print(f"Current chunk: {event_counter/interval}")
order = [[0,1], [1,2], [2,3], [3,4]] # Order what you want to align
max_process_event_chunk = 100 # End the loop early
processedEvents = 0 # Initialisation
initial_event_chunk = fReader.get_aligned_events(order=order, interval=interval)
#Initialisation
tdc_mets = [[] for tdc in range(5)]
Tot_TDC_info = [[] for tdc in range(5)]
with tqdm(total=max_process_event_chunk*interval, desc="Processing Events", unit='Events') as pbar:
    while processedEvents < max_process_event_chunk*interval:
        processedEvents += interval
        event_chunk, tdc_met, TDC_info = fReader.get_aligned_events(order=order, interval=interval, extract_tdc_mets = True) 
        # get_aligned_events have a choice to output the tdc metric and tdc information by setting extract_tdc_mets to be True
        [tdc_mets[i].append(tdc_met[i]) for i in range(5) if tdc_met[i] != 0]
        [Tot_TDC_info[i].extend(TDC_info[i]) for i in range(5) if TDC_info[i]]
        pbar.update(interval)


Current chunk: 100.0


Processing Events: 100%|██████████| 10000/10000 [00:06<00:00, 1488.67Events/s]


In [65]:
import matplotlib.pyplot as plt
colors = ['blue', 'green', 'red', 'purple', 'orange']
fig, ax = plt.subplots(figsize=(10, 8))
for tdc in range(5):
    met = tdc_mets[tdc]
    binsx = [event_counter + x*250 for x in range(len(met))]
    ax.plot(binsx ,met, label = f'tdc {tdc}', color = colors[tdc])

ax.set_xlim(event_counter,max_process_event_chunk*interval+event_counter)
# ax.set_ylim(0,1)
ax.legend()
ax.set_title('TDC monitoring metric')
ax.set_ylabel('bad time behavior / nominal time behavior')
ax.set_xlabel('processed Event')
plt.show()

By plotting these out, one will see the first hit time and first hit channels vary between the good and bad regions

In [67]:
importlib.reload(TTools)
TTools.plot_tdc_error_times(Tot_TDC_info)
TTools.plot_tdc_error_times_custom_ranges(Tot_TDC_info, [(0, 100), (100, 200)], output_pdf='TDC_first_hit_time.pdf')
TTools.plot_tdc_error_channels(Tot_TDC_info)
TTools.plot_tdc_error_channels_custom_ranges(Tot_TDC_info, [(0, 100), (100, 200)], tdcs_to_plot=None, output_pdf='TDC_first_hit_time_channel.pdf')

KeyboardInterrupt: 

Getting the variables used to calculate the alignment metric is even more difficult, since the output data is not representative to what the code used to compute the alignment metric, hence we need decorators to capture the variable in question during the program's run time, record them, and plot them. Below is an example decorator designed to capture the event words used when doing realignments. 

Essentially it access the function in question, capturing its' input and output, and calculating the the min channels after the original calculation. 

You may ask why do I do this, well this is because the functino calculating this is step 4 deep in the code, you need to pretty much add this extra check in every functino it passes through, eventually extracting it to the top level. And you probably shouldn't mess with fReader that much..

In [15]:
class Capturer:
    def __init__(self):
        self.TDC_alignment_time = [[] for _ in range(5)]
        self.processedEvents = 0 

    def extra_calculation_decorator(self, func):
        def wrapper(*args, **kwargs):
            minTimes = [300, 300]
            minChans = [-1, -1]
            minRPC = [-1, -1]
            minWord = [-1, -1]
            tdc = [-1, -1]
            eta = [True, True]
            
            rpc1Hits = args[0]
            rpc2Hits = args[1]
            skipChans = kwargs.get('skipChans', [])

            result = func(*args, **kwargs)
            
            if result == -1:
                return result
            for hit in rpc1Hits:
                if hit.time < minTimes[0] and hit.channel not in skipChans:
                    minTimes[0] = hit.time
                    minChans[0] = hit.channel
                    minRPC[0] = hit.rpc
                    eta[0] = hit.eta
            for hit in rpc2Hits:
                if hit.time < minTimes[1] and hit.channel not in skipChans:
                    minTimes[1] = hit.time
                    minChans[1] = hit.channel
                    minRPC[1] = hit.rpc
                    eta[1] = hit.eta
            
            if -1 in minChans:
                return -1
            
            a = TTools.rpcHitToTdcChan(minRPC[0], minChans[0], eta[0])
            b = TTools.rpcHitToTdcChan(minRPC[1], minChans[1], eta[1])
            
            tdc[0], minWord[0] = a
            tdc[1], minWord[1] = b

            self.TDC_alignment_time[tdc[0]].append((minTimes[0], a, self.processedEvents))
            self.TDC_alignment_time[tdc[1]].append((minTimes[1], b, self.processedEvents))

            return result

        return wrapper

In [7]:
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
importlib.reload(TTools)

Capturer = Capturer

# Apply the decorator
original_testAlign = ATools.testAlign
ATools.testAlign = Capturer.extra_calculation_decorator(ATools.testAlign)

# Main loop
interval = 100  # Set your monitoring chunk size
fReader = rawFileReader.fileReader(file_path[0])  # Load in the class object
order = [[0, 1], [1, 2], [2, 3], [3, 4]]  # Order what you want to align
max_process_event_chunk = 200  # End the loop early
processedEvents = 0  # Initialization
initial_event_chunk = fReader.get_aligned_events(order=order, interval=interval)
tdc_mets = [[] for tdc in range(5)]
Tot_TDC_info = [[] for tdc in range(5)]

with tqdm(total=max_process_event_chunk, desc="Processing Events", unit='Events') as pbar:
    while processedEvents < max_process_event_chunk:
        processedEvents += 1
        Capturer.processedEvents = processedEvents
        event_chunk = fReader.get_aligned_events(order=order, interval=interval)
        pbar.update(1)
# Return to original, or restart kernal
ATools.testAlign = original_testAlign



NameError: name 'Capturer' is not defined

Hence you can find which channels and which timing are used for alignment metric calculation

In [22]:
importlib.reload(TTools)
TTools.plot_tdc_alignment_channels_custom_ranges(Capturer.TDC_alignment_time, [(0, 50), (50, 100)], output_pdf='output/TDC_alignment_channels_used.pdf' )
TTools.plot_tdc_alignment_times_custom_ranges(Capturer.TDC_alignment_time, [(0, 50), (50, 100)], output_pdf='output/TDC_alignment_time_used.pdf' )

### time walk effect

It was found that signals takes time to travel from the FEB to the TDC, and on top of that, a systematic timing offset on each channel was also observed. Below is a tool written by Michael to read the timing difference between eta and phi channels, and averaging them across the entire chunk, and finally plotting them.

In [10]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
interval = 100 # Set your monitoring chunck size
fReader = rawFileReader.fileReader(file_path[0]) # load in the classs object
order = [[0,1], [1,2], [2,3], [3,4]] # Order what you want to align
max_process_event_chunk = 1000 # End the loop early
processedEvents = 0 # Initialisation
initial_event_chunk = fReader.get_aligned_events(order=order, interval=interval)
TAnalyser = proAnubis_Analysis_Tools.Timing_Analyser(initial_event_chunk,processedEvents)
while processedEvents < max_process_event_chunk:
    processedEvents += 1
    event_chunk = fReader.get_aligned_events(order=order, interval=interval)
    if event_chunk:
        TAnalyser.update_event(event_chunk, processedEvents)
        TAnalyser.readTDCTimeDiffs()
        
outDict = {'totDiffs':TAnalyser.totDiffs,
                    'nDiffs':TAnalyser.nDiffs,
                    'diffHists':TAnalyser.diffHists} 

The residual is calculated by applying the time walk correction, then averaging across the whole strip to a 2D plane. You should be able to see the time walk effect gone after applying the correction, and we are left with systematic corrections only

In [11]:
residEta, residPhi = TAnalyser.Calculate_Residual_and_plot_TDC_Time_Diffs( 
                                                     pdf_filename='output/TDC_time_diffs.pdf', 
                                                     max_itr = 5)

If you look at TAnalyser.`Calculate_Residual_and_plot_TDC_Time_Diffs`, there is a magic number for slope and offset, which is curve fitted by the function below. feel free to change it after fitting a larger amount of data set

In [9]:
importlib.reload(proAnubis_Analysis_Tools)
TAnalyser.plot_and_fit_tof_corrections()

0.15415730778671458 16.159064658832435
R² value: 0.6248481250576245


To look at each individual strip, one can use this function

In [27]:
importlib.reload(TTools)
TTools.show_strip_time_info(outDict, 20, 22, 2)

You can also plot the residual through this function

In [28]:
importlib.reload(proAnubis_Analysis_Tools)
TAnalyser.plot_residual()

### Conclusion

The Timing analysis class contains all the functions needed to analyse the time walk effect, capturing timing corruptions in TDCs, and also monitoring tdcs as well. A general idea is that anything designed to run smoothly to produce the most useful result is written in TAnalyser with all instances stored internally. Analysis on individual bits will be in TTools. 

# Reconstructor
Very similar to the Timing Analyser, the Reconstructor's main goal is to reconstruct muon paths from the given data. The details of the code can be found under the documentation under the function RTools.`reconstruct_timed_Chi2_ByRPC`

below is an example of finding the efficiency of each RPC. The test RPC is removed, and a line is reconstructed using other 5 RPCs. This line is then extrapolated to the test RPCs' location, and a area of certain radius called the `tolerance` is probed for the hits. the success and failure together with their location is recorded

In [14]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
importlib.reload(RTools)
interval = 100 # Set your monitoring chunck size
fReader = rawFileReader.fileReader(file_path[0]) # load in the classs object
print(file_path[0])
order = [[0,1], [1,2], [2,3], [3,4]] # Order what you want to align
max_process_event_chunk = 1_000 # End the loop early
processedEvents = 0 # Initialisation
initial_event_chunk = fReader.get_aligned_events(order=order, interval=interval)
reconstructor = proAnubis_Analysis_Tools.Reconstructor(initial_event_chunk, processedEvents)
with tqdm(total=max_process_event_chunk, desc="Processing Events", unit='Events') as pbar:
    while processedEvents < max_process_event_chunk:
        processedEvents += 1
        try:
            event_chunk = fReader.get_aligned_events(order=order, interval=interval)
        except:
            print(f'Error at event chunk {processedEvents}')
            continue
        #Zone of Reconstruction
        if event_chunk:
            # We need to update the event like TAnalyser as well
            reconstructor.update_event(event_chunk, processedEvents)
            # populate_hits turns TDC bit wise information into their corresponding strips
            reconstructor.populate_hits()
            # This is optionnal, and requires the residual of eta and phi
            reconstructor.apply_systematic_correction(residEta, residPhi)
            # make_cluster does temporal and spatial coincidence between the stips, and reconstruction is done
            cluster = reconstructor.make_cluster()
            #print("Clust")
            print(cluster)
            # Finally, recontruction is done using cluster information
            reconstructor.reconstruct_and_extrapolate(cluster)
        pbar.update(1)

C://Users//jony//Programming//Python//Anubis//anubis////data//proAnubis_240818_1925.raw


Processing Events:   0%|          | 1/1000 [00:00<02:34,  6.46Events/s]

[[1, np.float64(193.10008370402574), [[[[[0, 15, np.float64(178.72578417965047), False]]], [[[0, 18, np.float64(193.10008370402574), True]]]], [[[[1, 15, np.float64(179.45316537381828), False]]], [[[1, 18, np.float64(194.96307165709345), True]]]], [[[[2, 15, np.float64(178.78177177479284), False]]], [[[2, 18, np.float64(195.28283606267286), True]]]], [[[[3, 13, np.float64(182.22712097633814), False], [3, 14, np.float64(183.56297271981364), False]]], []], [[], []], [[[[5, 12, np.float64(187.45515724191267), False]]], [[[5, 1, np.float64(200.7251867211328), True], [5, 2, np.float64(201.0487343071976), True]]]]]], [2, np.float64(193.47155606908078), [[[[[0, 28, np.float64(186.84917407064057), False], [0, 29, np.float64(184.71490189102846), False], [0, 30, np.float64(185.6651923273582), False]]], [[[0, 15, np.float64(197.33354379018385), True], [0, 16, np.float64(193.47155606908078), True]]]], [[], []], [[[[2, 29, np.float64(184.96559674117236), False]]], [[[2, 16, np.float64(195.282961709

Processing Events:   0%|          | 2/1000 [00:00<02:30,  6.62Events/s]

[[3, np.float64(205.94881043715867), [[[[[0, 27, np.float64(195.50735192136412), False]]], [[[0, 2, np.float64(206.079511983203), True]]]], [[[[1, 27, np.float64(196.70017679395232), False]]], [[[1, 2, np.float64(205.94881043715867), True]]]], [[], [[[2, 2, np.float64(207.13975933050162), True]]]], [[], []], [[], []], [[[[5, 14, np.float64(196.5651874989854), False], [5, 15, np.float64(197.35794923287162), False]]], [[[5, 16, np.float64(212.62146284859682), True], [5, 17, np.float64(211.5295133693287), True]]]]]], [4, np.float64(194.41268532458452), [[[[[0, 14, np.float64(186.39007587881315), False]], [[0, 16, np.float64(184.7869064580271), False], [0, 17, np.float64(184.60030848929352), False], [0, 18, np.float64(184.24071328264588), False], [0, 19, np.float64(183.49861679225427), False], [0, 20, np.float64(184.01824968395596), False], [0, 21, np.float64(182.50956737587995), False], [0, 22, np.float64(181.4409800366395), False], [0, 23, np.float64(182.06824405792548), False], [0, 24, 

...

Processing Events:   1%|          | 8/1000 [00:01<03:04,  5.38Events/s]

[[8, np.float64(194.11168656606193), [[[[[0, 35, np.float64(185.9955423903249), False], [0, 36, np.float64(183.89222574162605), False], [0, 37, np.float64(186.49305798283567), False]]], [[[0, 21, np.float64(194.79184959710108), True], [0, 22, np.float64(196.16887213487215), True]]]], [[[[1, 34, np.float64(186.9592193929535), False], [1, 35, np.float64(185.3647400050812), False], [1, 36, np.float64(188.12499465319524), False]]], [[[1, 21, np.float64(196.57002763791104), True], [1, 22, np.float64(198.47818316526855), True]]]], [[[[2, 35, np.float64(184.7810713125661), False]]], []], [[[[3, 41, np.float64(190.4718441142642), False]]], [[[3, 2, np.float64(196.26700595079257), True], [3, 3, np.float64(194.11168656606193), True]]]], [[], []], [[], []]]], [9, np.float64(194.50656487732806), [[[[[0, 22, np.float64(193.9409800366395), False], [0, 23, np.float64(183.63074405792548), False], [0, 24, np.float64(183.64827514459373), False], [0, 25, np.float64(184.86781785143071), False]]], [[[0, 17

Processing Events:   1%|          | 9/1000 [00:01<03:11,  5.17Events/s]


[[10, np.float64(193.23204849401773), [[[[[0, 17, np.float64(181.47530848929352), False]]], [[[0, 8, np.float64(193.23204849401773), True]]]], [[], [[[1, 8, np.float64(194.11143254349548), True]]]], [[[[2, 17, np.float64(183.33100009173415), False]]], [[[2, 7, np.float64(195.3681564037521), True]]]], [[[[3, 14, np.float64(189.81297271981364), False]]], []], [[], []], [[], []]]], [11, np.float64(205.17060323424406), [[[[[0, 11, np.float64(190.51536090979135), False]]], [[[0, 13, np.float64(205.51160567389167), True]]]], [[[[1, 11, np.float64(190.86379619688583), False]]], [[[1, 13, np.float64(205.17060323424406), True]]]], [[[[2, 11, np.float64(191.12428801063982), False]]], [[[2, 12, np.float64(205.1844101552697), True], [2, 13, np.float64(205.70312487113625), True]]]], [[[[3, 8, np.float64(196.04694233591374), False], [3, 9, np.float64(194.16415769568513), False]]], [[[3, 6, np.float64(205.76392417590114), True]]]], [[[[4, 6, np.float64(199.53665227005115), False]]], []], [[[[5, 6, np

KeyboardInterrupt: 

Plotting the efficiency

In [13]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
for RPC in range(6):
    if reconstructor.possible_reconstructions[RPC] == 0:
        efficiency = [0 for x in reconstructor.successful_reconstructions[RPC]]
    else:
        efficiency = [x / reconstructor.possible_reconstructions[RPC] for x in reconstructor.successful_reconstructions[RPC]]
    plt.plot(reconstructor.tol, efficiency, label=f'RPC {RPC}')

plt.xlabel('Tolerance')
plt.ylabel('Efficiency')
plt.title('The efficiency of the reconstruction')
plt.legend()
plt.grid(True)
plt.show()
print(reconstructor.possible_reconstructions)

[4618, 4201, 4838, 4592, 4747, 6202]


You can also plot a heat map using the information obtained from the reconstruction

In [31]:
import matplotlib.colors as colors
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages
rpcNames = {0: "Triplet Low", 1: "Triplet Mid", 2: "Triplet Top", 3: "Singlet", 4: "Doublet Low", 5: "Doublet Top"}

success_events = [[0 for etchan in range(32)] for phchan in range(64)]

with PdfPages('output/reconstruction_heatmap_plots.pdf') as pdf:
    for rpc in range(6):
        for ph in range(64):
            for et in range(32):
                if reconstructor.successful_reconstructed_coords[rpc][ph][et] > 0:
                    total_successful = reconstructor.successful_reconstructed_coords[rpc][ph][et]
                    total_events = reconstructor.possible_reconstructions_coords[rpc][ph][et]
                    if total_events > 0:
                        success_events[ph][et] = total_successful / total_events
                    else:
                        success_events[ph][et] = 0  # No events, efficiency is 0

        fig, ax = plt.subplots(1, figsize=(16, 8), dpi=100)
        etachannels = [x - 0.5 for x in range(33)]
        phichannels = [x - 0.5 for x in range(65)]
        etaHist = (success_events, np.array(phichannels), np.array(etachannels))
        zrange = [0, max(max(row) for row in success_events)]
        thisHist = hep.hist2dplot(etaHist, norm=colors.Normalize(zrange[0], zrange[1]))
        thisHist.cbar.set_label('Successful reconstructions / Possible reconstructions', rotation=270, y=0.3, labelpad=23)
        plt.ylim(31.5, -0.5)
        plt.ylabel("Eta Channel")
        plt.xlabel("Phi Channel")
        ax.set_title(rpcNames[rpc])

        # Draw lines
        x_points = [-0.5, 64.5]
        y_points = [7.5, 15.5, 23.5]
        for y_point in y_points:
            plt.plot(x_points, [y_point, y_point], 'k', linestyle='dotted')
        y_points = [-0.5, 31.5]
        x_points = [7.5, 15.5, 23.5, 31.5, 39.5, 47.5, 55.5]
        for x_point in x_points:
            plt.plot([x_point, x_point], y_points, 'k', linestyle='dashed')

        pdf.savefig(fig)
        plt.close(fig)

print("PDF created successfully.")

PDF created successfully.


### Extracting angle information

One side product of reconstruction algorithm is the extraction of angles. This uses all 6 RPCs to reconstruct tracks, and find their angular distribution. The reason why a separate function is used to the efficiency calculation will be made clear later in the detailed explaination section

In [12]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
interval = 100 # Set your monitoring chunck size
fReader = rawFileReader.fileReader(file_path[-1]) # load in the classs object
order = [[0,1], [1,2], [2,3], [3,4]] # Order what you want to align
max_process_event_chunk = 150 # End the loop early
processedEvents = 0 # Initialisation
initial_event_chunk = fReader.get_aligned_events(order=order, interval=interval)
reconstructor = proAnubis_Analysis_Tools.Reconstructor(initial_event_chunk, processedEvents, tof_correction=True)
TAnalyser = proAnubis_Analysis_Tools.Timing_Analyser(initial_event_chunk,processedEvents)
with tqdm(total=max_process_event_chunk, desc="Processing Events", unit='Events') as pbar:
    while processedEvents < max_process_event_chunk:
        processedEvents += 1
        event_chunk = fReader.get_aligned_events(order=order, interval=interval)
        if event_chunk:
            reconstructor.update_event(event_chunk, processedEvents)
            # if processedEvents < 250:
            #     pbar.update(1)
            #     continue
            reconstructor.populate_hits()
            #reconstructor.apply_systematic_correction(residEta, residPhi)
            cluster = reconstructor.make_cluster()
            filtered_events = RTools.filter_events(cluster,1,6)     
            reconstructor.extract_angles_phi_eta_timed_DZ_modified(filtered_events)
        pbar.update(1)

Processing Events: 100%|██████████| 150/150 [00:18<00:00,  8.32Events/s]


I was about to make this into a function, but then i need to write documentation for a code purely for plotting, might as well write it out here

In [13]:
import matplotlib.pyplot as plt
import numpy as np
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(14, 20))
bin_edges = np.arange(-90.5, 91.5, 1)
phi_edges = np.arange(-180.5, 181.5, 1)
ax1.bar(bin_edges[:-1], reconstructor.eta_histogram, width=1, edgecolor='black', align='edge')
ax1.set_title('eta Angles Histogram chunk3')
ax1.set_xlabel('eta Angle (degrees)')
ax1.set_ylabel('Counts')

ax2.bar(bin_edges[:-1], reconstructor.phi_histogram, width=1, edgecolor='black', align='edge')
ax2.set_title('phi Angles Histogram chunk3')
ax2.set_xlabel('phi Angle (degrees)')
ax2.set_ylabel('Counts')

ax3.bar(phi_edges[:-1], reconstructor.solid_theta_histogram, width=1, edgecolor='black', align='edge')
ax3.set_title('solid theta Angles Histogram chunk3')
ax3.set_xlabel('solid theta Angle (degrees)')
ax3.set_ylabel('Counts')

ax4.bar(phi_edges[:-1], reconstructor.solid_phi_histogram, width=1, edgecolor='black', align='edge')
ax4.set_title('solid phi Angles Histogram chunk3')
ax4.set_xlabel('solid phi Angle (degrees)')
ax4.set_ylabel('Counts')


plt.tight_layout()
plt.show()

## Time of Flight analysis

Now we can do the recontruction and find the time of flight information across RPCs to determine our time resolution and also potential errors. This is another function called `reconstruct_and_findtof`, which records the hit time difference of the reconstructed paths by specifying which set of RPC one needs to compare

In [42]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
interval = 100 # Set your monitoring chunck size
fReader = rawFileReader.fileReader(file_path) # load in the classs object
order = [[0,1], [1,2], [2,3], [3,4]] # Order what you want to align
rpc_comparison = [[0,1], [0,2], [0,3], [0,4], [0,5]]
max_process_event_chunk = 150 # End the loop early
processedEvents = 0 # Initialisation
initial_event_chunk = fReader.get_aligned_events(order=order, interval=interval)
reconstructor = proAnubis_Analysis_Tools.Reconstructor(initial_event_chunk, processedEvents, tof_correction=True)
with tqdm(total=max_process_event_chunk, desc="Processing Events", unit='Events') as pbar:
    while processedEvents < max_process_event_chunk:
        processedEvents += 1
        event_chunk = fReader.get_aligned_events(order=order, interval=interval)
        if event_chunk:
            reconstructor.update_event(event_chunk, processedEvents)
            # if processedEvents < 250:
            #     pbar.update(1)
            #     continue
            reconstructor.populate_hits()
            reconstructor.apply_systematic_correction(residEta, residPhi)
            cluster = reconstructor.make_cluster()
            reconstructor.reconstruct_and_findtof(cluster, rpc_comparisons=rpc_comparison)
        pbar.update(1)

Processing Events: 100%|██████████| 150/150 [00:28<00:00,  5.32Events/s]


In [43]:
importlib.reload(RTools)
RTools.compile_and_plot_tof(reconstructor.dT,rpc_comparison, pdf_filename='Data_output/tof.pdf')

Mid average value for RPC0-5: 1.3987860461696622
Gaussian fit parameters for RPC[0, 1]: amplitude = 53.50691046124199, mean = 1.2048419127332568, std deviation = 0.9790481241399833
Mid average value for RPC1-5: 0.8784803476790051
Gaussian fit parameters for RPC[0, 2]: amplitude = 53.7484141009169, mean = 0.6984959538885683, std deviation = -0.9981717275185302
Mid average value for RPC2-5: 2.6366421505285005
Gaussian fit parameters for RPC[0, 3]: amplitude = 52.43693916600671, mean = 2.357014405626686, std deviation = 0.8905643701367584
Mid average value for RPC3-5: 4.211738729375389
Gaussian fit parameters for RPC[0, 4]: amplitude = 43.36090025141649, mean = 5.095888569337571, std deviation = 1.0992980633930405
Mid average value for RPC4-5: 4.84941114066617
Gaussian fit parameters for RPC[0, 5]: amplitude = 44.0236360382068, mean = 5.349853481709834, std deviation = 1.0639506859991157


'Data_output/tof.pdf'

In [44]:
importlib.reload(RTools)
RTools.compile_and_plot_tof_chunk(reconstructor.dT,rpc_comparison, 10, pdf_filename='Data_output/tof_chunks.pdf')

Mid average value for RPC[0, 1], chunk1: 1.024887145104885
Gaussian fit parameters for RPC0-1: amplitude = 6.058657696836265, mean = 0.994628386001923, std deviation = -0.8725887113917731
Mid average value for RPC[0, 1], chunk2: 1.396897318093764
Gaussian fit parameters for RPC0-2: amplitude = 8.458237150315759, mean = 1.1614738324206986, std deviation = -0.4922944291647427
Mid average value for RPC[0, 1], chunk3: 1.346201154458408
Gaussian fit parameters for RPC0-3: amplitude = 4.530939846704952, mean = 1.341655804055145, std deviation = 1.1572258849713841
Mid average value for RPC[0, 1], chunk4: 1.2870266563429373
Gaussian fit parameters for RPC0-4: amplitude = 5.5814782660196975, mean = 1.4801366233995297, std deviation = -0.9433402364208442
Mid average value for RPC[0, 1], chunk5: 1.366932843589334
Gaussian fit parameters for RPC0-5: amplitude = 5.657049969015328, mean = 1.253895360273652, std deviation = 0.9672800854498189
Mid average value for RPC[0, 1], chunk6: 1.560018879318188

'Data_output/tof_chunks.pdf'

In [45]:
importlib.reload(proAnubis_Analysis_Tools)
reconstructor.plot_tof_offset(rpc_comparison)

c:\Users\jony\Programming\Python\Anubis\summer2\Tutorial with Osiris\.venv\lib\site-packages\numpy\_core\fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\jony\Programming\Python\Anubis\summer2\Tutorial with Osiris\.venv\lib\site-packages\numpy\_core\_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
